In [80]:
import numpy as np

In [91]:
%store -r ionosphere_headers
%store -r ionosphere_export

no stored variable ionosphere_data_headers
no stored variable ionosphere_data_export


In [82]:
%run naive_bayes.ipynb

In [83]:
# %run logistic_regression.ipynb

In [84]:
data = adult_data_export
data_headers = adult_data_headers
# data.shape

In [85]:
def get_new_model(model):
    if model == "LogisticRegression":
        return LogisticRegression(1000)
    else:
        return NaiveBayes()

In [86]:
def evaluate_acc(y,yh):
    tp = np.sum(np.logical_and(yh==1,y==1))
    tn = np.sum(np.logical_and(yh==0,y==0))
    
    acc = (tp+tn)/y.size
    return acc

In [87]:
def get_folds(k, data):
    np.random.shuffle(data)
    
    # Split into k-folds and remove empty splits if not appropriate size
    folds = np.array_split(data, k)
    folds = [x for x in folds if x.size > 0]
    
    return folds

In [88]:
def k_fold_cross_validation(k, data, model_name="LogisticRegression", data_headers=[]):
    folds = get_folds(k,data)
    fold_accuracies = []
    for i in range(k):
        print(i)
        model = get_new_model(model_name)
        
        # Get test row, and training set
        test = folds[i]
        train = np.delete(folds, i, 0)
        
        train = np.concatenate(train,axis=0)
        train = np.array(train)
        
        train_X = train[:, :-1]
        train_Y = train[:, -1]
        
        # Fit and predict
        if(model_name == "NaiveBayes"):
            model.set_headers(data_headers)
        model.fit(train_X, train_Y)
        
        test_X = test[:, :-1]
        test_Y = test[:, -1]
        model_results = model.predict(test_X)
        
        fold_accuracies.append(evaluate_acc(model_results,test_Y))
    return np.average(fold_accuracies)

In [90]:
k_fold_cross_validation(10,data, model_name="NaiveBayes", data_headers=data_headers)

0
(40699,)


KeyboardInterrupt: 

In [68]:
# # np.random.shuffle(adult_data_export)

# train_X = ionosphere_export[:300, :-1]
# train_y = ionosphere_export[:300, -1]

# test_X = ionosphere_export[300:, :-1]
# test_y = ionosphere_export[300:, -1]

# lr = LogisticRegression(100000)
# lr.fit(train_X, train_y)

# y_pred = lr.predict(test_X)
# lr1_acc = evaluate_acc(y_pred, test_y)


In [69]:
# lr2 = linear_model.LogisticRegression()
# lr2.fit(train_X, train_y)

# print(lr2.coef_) # returns a matrix of weights (coefficients)'

# y_pred = lr2.predict(test_X)
# lr2_acc = evaluate_acc(y_pred, test_y)

# print("lr1: ", lr1_acc)
# print("lr2: ", lr2_acc)
